In [1]:
!pwd

/home/lang-grouping


In [4]:
import torch
dump= torch.load("snapshot_bw.pt") # colors_precomp 비어있음
dump[16]

tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int32)

In [8]:
len(dump[2])

50613

In [5]:
len(dump[16])

50613

In [11]:
dump[17].shape #sh obj

torch.Size([50613, 1, 16])

In [13]:
dump[16].shape #dl out obj

torch.Size([50613])

In [14]:
dump[15].shape #sh

torch.Size([50613, 16, 3])

In [17]:
dump[4].shape

torch.Size([1])

In [9]:
len(dump[17])

50613